# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [12]:
#import libraries:
import numpy as np 
from sklearn.preprocessing import normalize #normalizing gives better results. Experiment with this
import cv2
import open3d as o3d

In [13]:
def read_transformations(filename='poses.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    transformation_list = []
    for i in range(len(lines)):
        transformation_list_temp = lines[i].split()
        temp_rot = [] 
        temp_rot.append( (transformation_list_temp[0:4] ) ) 
        temp_rot.append( (transformation_list_temp[4:8]  ) ) 
        temp_rot.append( (transformation_list_temp[8:12]  ) ) 
        transformation_list.append(temp_rot)
    return transformation_list
   

def newpoint(dp3, x):
    
    x1 = np.array([
        [x[0][0], x[0][1], x[0][2], x[0][3]],
        [x[1][0], x[1][1], x[1][2], x[1][3]],
        [x[2][0], x[2][1], x[2][2], x[2][3]],
    ])
    pts = np.mat([dp3[0], dp3[1], dp3[2], 1]).T
    mats = np.matmul(x1, pts)
    return mats


Provide explanation in this cell: 


In [14]:
# your code here 
'''
The total number of point clouds that we will recieve is 
There are 40 cameras (or 20 stereo camera)
'''

# Number of pixels in all 40 images. 

calibF=open('calib.txt')
line=0
for lines in calibF:
    if(line==1):
        k= np.array(lines.strip('\n').split(' '))
    if(line==4):
        bDash=lines.split(' ')
        b=float(bDash[0])
    line+=1
kDash=k.astype('float64').reshape(3,3)

worldPoints=[]
pose_sh = []
transformationList=np.array(read_transformations()).astype('float64')
pose0 = np.vstack([transformationList[0], np.array([0, 0, 0, 1])])
pose_sh.append(pose0)
for i in range(1, len(transformationList)):
    posey = np.vstack([transformationList[i], np.array([0, 0, 0, 1])])
    pose_sh.append(posey)


# calcuation of Q matrix
    
import matplotlib.pyplot as plt
pathL='img2/'
pathR='img3/'

fl=7.070912e+02
bl = 0.53790448812

xc=185 # Centre pixel
yc=613 # Centre pixel 

# calculation of Q m
f=k[0][0]



# imgLrGB = cv2.cvtColor('/img2/0000000460.png', cv2.COLOR_BGR2RGB)
# grayRrGB = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)

camPoints=[]
colours=np.zeros((370, 1226, 3))
thdoutputs=[]
thdcolours=[]
imgPointMap = {}
imgPointMap_alt = {}
imgPointMap_2 = {}

NumOfPairs=20

for i in range(NumOfPairs):
    img2=pathL+'0'*7+str(460+i)+str('.png')
    img3=pathR+'0'*7+str(460+i)+str('.png')
    

    imgLF=cv2.imread(img2)
    imgRF=cv2.imread(img3)
    
    imgL=cv2.cvtColor(imgLF,cv2.COLOR_BGR2RGB)
    imgR=cv2.cvtColor(imgRF,cv2.COLOR_BGR2RGB)
    
    scale_percent = 100 # percent of original size
    width = int(imgL.shape[1] * scale_percent / 100)
    height = int(imgR.shape[0] * scale_percent / 100)
    dim = (width, height) 
    h, w = imgL.shape[:2]
    print(h)
    numOfPix= width*height
    Q=np.array([[1,0,0,-w/2],[0,-1,0,h/2],[0,0,0,fl],[0,0,-bl**(-1),0]])
    Q=Q.astype('float64')
    
#     imgL=cv2.resize(imgLF, dim, interpolation = cv2.INTER_jnreconAREA) 
#     imgR=cv2.resize(imgRF, dim, interpolation = cv2.INTER_AREA) 

    window_size = 3 
    
    minDisp = 16
    numDisp = 112 - minDisp
    
    ImageDisp = cv2.StereoSGBM_create(
        minDisparity=minDisp,             
        blockSize=7,
        numDisparities=numDisp + (2) * 16,
        P1=8*3*window_size**2,    
        P2=32*3*window_size**2,
        uniquenessRatio=12,
        speckleWindowSize=400,
        disp12MaxDiff=1,
        speckleRange=5,
        #preFilterCap=63
    )
    
    
    dispMat = ImageDisp.compute(imgL, imgR).astype(np.float32)
    print(dispMat)
    
   
   
    
           
    # cv2 way of finding 3d points
    camPoints = cv2.reprojectImageTo3D(dispMat, Q)
    colours = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)
    
    
    imgPointMap[i] = []
    imgPointMap_alt[i] = []
    imgPointMap_2[i] = []
    for k in range(len(imgL)):
        for j in range(len(imgL[k])):
            colours[k][j] = imgL[k][j]
            pt_temp = Q @ np.array([k, j, dispMat[k][j]/16.0, 1]).T
            W = pt_temp[3]
            to_ap = [pt_temp[0]/W, pt_temp[1]/W, pt_temp[2]/W]
            camPoints[k][j] = to_ap
            imgPointMap[i].append(([k, j], to_ap))
            if dispMat[i][j] > dispMat.min():
                imgPointMap_alt[i].append(([k, j], to_ap))
                # This step takes the above project 3d, and transforms them according to the ground truth poses.
                pt_new_temp = np.array(newpoint(to_ap, pose_sh[i]).T).astype('float32')[0]
                pt_add = [pt_new_temp[0], pt_new_temp[1], pt_new_temp[2]]
                imgPointMap_2[i].append(([k, j], pt_add))

    mask_map = dispMat > dispMat.min()
    o_camPoints = camPoints[mask_map]
    o_colours = colours[mask_map]
    o_colours =(o_colours/255).astype('float64')
    for j,pts in enumerate(o_camPoints):
        thdoutputs.append(np.array(newpoint(pts, pose_sh[i]).T).astype('float32')[0])
        thdcolours.append(o_colours[j])
        
thdoutputs = np.array(thdoutputs)
thdcolours = np.array(thdcolours)

#print(thdoutputs)
# print(thdcolours.shape)

accPntCld=[]
pntCld= o3d.geometry.PointCloud()
pntCld.points= o3d.utility.Vector3dVector(thdoutputs)
pntCld.colors= o3d.utility.Vector3dVector(thdcolours)
o3d.io.write_point_cloud("all_pcs.ply", pntCld)
o3d.visualization.draw_geometries([pntCld])
   

370
[[240. 240. 240. ... 240. 240. 240.]
 [240. 240. 240. ... 240. 240. 240.]
 [240. 240. 240. ... 368. 240. 240.]
 ...
 [240. 240. 240. ... 240. 240. 240.]
 [240. 240. 240. ... 240. 240. 240.]
 [240. 240. 240. ... 240. 240. 240.]]


In [15]:
# temp=colours
# o3d.visualization.draw_geometries([pntCld])
# img=np.array(cv2.imread('img2/'+'0'*7+str(460+0)+str('.png')))
# print(img.shape)

In [16]:
# ##############-------Obtaining world points............######


# pt_new_temp = np.array(make_new_pt(to_ap, combination[count+start]).T).astype('float32')[0]
# for i in range(NumOfPairs):
#      worldPoints.append(transformationList[i].dot(camPoints[i]))

# # worldPoints=np.array(worldPoints)

# # siz=worldPoints.shape
# # worldPoints=worldPoints.reshape(siz[1],siz[0]*siz[2])


# # colours=temp
# # colours=colours.reshape(siz[0]*siz[2],siz[1])
# # print(colours.shape)

# # # temp=np.ones((1,worldPoints.shape[2]))

# # # np.row_stack((worldPoints[0],
# # #           temp))

# # # print(worldPoints[0].shape)
 
# # # # for i in range(worldPoints.shape[0]):
# # # #     np.concatenate((worldPoints[i],
# # # #                np.ones((406260))),axis=1)


# # # mmk commented below

# # accPntCld=[]
# # pntCld= o3d.geometry.PointCloud()
# # pntCld.points= o3d.utility.Vector3dVector(worldPoints.T)
# # pntCld.colors= o3d.utility.Vector3dVector(colours)
# # o3d.io.write_point_cloud("all_pcs.ply", pntCld)
# # o3d.visualization.draw_geometries([pntCld])

# #mmk commented


# #     for x in range(height):
# #         for y in range(width):
# #             XL=(b/disparity[x-xc])*imgL[i,:2]
# #             XL.append()
# #             XR=(b/disparity[y-yc])*imgR[i,:2]
# #     camPoints.append(XL)
# #     camPoints.append(XR)
    
    
# #   tempzero[:,0]=disparity
    
# #     disparity=tempz
    
# #     wrld= b*
    
# # print(dispMat.shape)

----
### PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>